## Load Mask RCNN into Google Colab

In [0]:
!git clone https://github.com/matterport/Mask_RCNN

## Set up the GPU Environment
In Colabs go to:  
Edit > Notebook Settings  
Set the hardware accelerator to 'GPU' and reload the notebook. 

In [0]:
#RUN GPU
%tensorflow_version 1.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Move into the Mask RCNN directory and install the package

In [0]:
import os
os.chdir('Mask_RCNN')

In [0]:
!ls

In [0]:
!python3 setup.py install;

## Load Modules

In [0]:
import mrcnn
import mrcnn.model as modellib
from mrcnn.config import Config
from mrcnn import utils
from mrcnn.model import MaskRCNN

import numpy as np
import colorsys
import argparse
import random
import os
import sys
import time
import json
import skimage
import datetime
from keras.models import load_model


## Link Google Drive and set up directories

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
homedir='/content/drive/My Drive/Colab Notebooks'
datadir=homedir+'/RCNN_test/'

In [0]:
#files need to be accessed using os !ls does not work.
os.listdir(homedir)

## Define the Config and Dataset classes

In [0]:
class myMaskRCNNConfig(Config):
    NAME = "MaskRCNN_config"
 
    # set the number of GPUs to use along with the number of images
    # per GPU
    # Colabs provides a 12GB GPU which should fit 2 images
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2
 
    # number of classes (we would normally add +1 for the background)
    # tree + BG
    NUM_CLASSES = 1+1
   
    # Number of training steps per epoch (set to number of training samples)
    STEPS_PER_EPOCH = 80
    
    # Learning rate (reduce to stop NaN losses)
    LEARNING_RATE=0.001
    
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    
    # setting Max ground truth instances
    MAX_GT_INSTANCES=10
    
config = myMaskRCNNConfig()
config.display()

check=[]
class TreeDataset(utils.Dataset):

    def load_dataset(self, dataset_dir):
        """
        Load the images and annotations from Google Drive
        The mask coordinates are save in 'polygons'
        """

        self.add_class("tree", 1, "tree")

        # Load annotations
        # VGG Image Annotator saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        annotations = json.load(open(os.path.join(homedir, "treeData.json")))
        annotations = list(annotations.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        for a in annotations:
            if a['filename'] in os.listdir(dataset_dir):
                # Get the x, y coordinates of points of the polygons that make up
                # the outline of each object instance. These are stored in the
                # shape_attributes (see json format above)
        
                if type(a['regions']) is dict:
                        polygons = [r['shape_attributes'] for r in a['regions'].values()]
                else:
                        polygons = [r['shape_attributes'] for r in a['regions']]
        
                # load_mask() needs the image size to convert polygons to masks.
                # Unfortunately, VIA doesn't include it in JSON, so we must read
                # the image. This is only managable since the dataset is tiny.
                image_path = os.path.join(dataset_dir, a['filename'])
                image = skimage.io.imread(image_path)
                height, width = image.shape[:2]
    
                self.add_image(
                    "tree",
                    image_id=a['filename'],
                    path=image_path,
                    width=width, height=height,
                    polygons=polygons)

    def load_mask(self, image_id):
        """
        Generate instance masks for an image.
        
        Returns:
        
        masks: A bool array of shape [height, width, instance count] with
               one mask per instance.
        
        class_ids: a 1D array of class IDs of the instance masks.
        
        """
        # If not a tree dataset image, delegate to background class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "tree":
            print('not a tree!')
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)

        class_id=[]
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1
            class_id.append(self.class_names.index('tree'))

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask, np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "tree":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

# Prepare the Training and Test Sets

In [0]:
# prepare train set
train_set = TreeDataset()
train_set.load_dataset(datadir+'/train')
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# prepare test/val set
test_set = TreeDataset()
test_set.load_dataset(datadir+'/test')
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

# Prepare the Training Model:

In [0]:
print("Loading Mask R-CNN model...")
model = modellib.MaskRCNN(mode="training", config=config, model_dir='./')

#load the weights for COCO
model.load_weights(homedir+'/mask_rcnn_coco.h5', 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

## Train the Model:
If you begin to encounter NaNs in your loss values, try lowering the learning rate by 4 or so times. 

In [0]:
# train heads with higher lr to speedup the learning

model.train(train_set, test_set, learning_rate=2*config.LEARNING_RATE, epochs=15, layers='heads')
history = model.keras_model.history.history

## Check the Output
Find the output directory name.  
In this example it is: 'maskrcnn_config20200218T1359'

In [0]:
!ls

In [0]:
#Move to the latest model directory
os.chdir('./maskrcnn_config20200424T1244')

## Save Model Weights and History in Google Drive:
Find the name of the latest model weights to save

In [0]:
!ls -lhrt

In [0]:
now_str='20200424T1244'

try:
  os.makedirs(homedir+'saved')
except:
  print('Save directory exists!')

weights_fname =homedir+'saved/weights_tree_'+now_str+'.h5'
history_fname =homedir+'saved/history_tree_'+now_str+'.json'

model.keras_model.save_weights(weights_fname)

with open(history_fname,"w") as f:
   f.write(json.dumps(str(history)))
   